In [5]:
import csv
import json
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

class QAImagePromptGenerator:
    def __init__(self):
        # Các mẫu phong cách khác nhau để lựa chọn
        self.style_templates = {
            "flat": "clean, flat illustration style with vibrant colors and simple background",
            "infographic": "educational infographic style with labeled diagrams and clear visual hierarchy",
            "cartoon": "friendly cartoon style with expressive characters and bright colors",
            "minimal": "minimalist design with essential elements only and subtle color palette",
            "realistic": "realistic illustration style with detailed rendering and natural lighting"
        }
        
        # Cấu trúc prompt cơ bản
        self.prompt_template = (
            "Create an educational illustration that explains: {input_text}. "
            "Style: {style_description}. "
            "The image should show {main_elements} with {visual_details}."
        )

    def extract_key_concepts(self, text):
        """
        Phân tích văn bản để trích xuất các khái niệm chính
        Đây là phiên bản đơn giản, trong ứng dụng thực tế bạn có thể 
        sử dụng NLP hoặc LLM để phân tích phức tạp hơn
        """
        # Giả lập quá trình trích xuất khái niệm - trong thực tế nên sử dụng
        # mô hình NLP hoặc ChatGPT API để phân tích tốt hơn
        words = text.lower().split()
        keywords = [word for word in words if len(word) > 3 and word not in [
            "tại", "sao", "vì", "khi", "qua", "hơn", "với", "của", "các", "đó", "này"
        ]]
        return list(set(keywords[:5]))  # Lấy tối đa 5 khái niệm không trùng lặp

    def generate_visual_elements(self, concepts):
        """Tạo mô tả về các yếu tố cần có trong hình ảnh dựa trên khái niệm"""
        # Đây là phiên bản đơn giản - trong ứng dụng thực tế sẽ phức tạp hơn
        elements = ", ".join(concepts)
        return f"core concepts including {elements}"

    def generate_prompt(self, question, answer, style="flat"):
        """Tạo prompt hoàn chỉnh từ câu hỏi và câu trả lời"""
        input_text = f'Câu hỏi: "{question}" Trả lời: "{answer}"'
        
        # Trích xuất khái niệm từ cả câu hỏi và câu trả lời
        concepts = self.extract_key_concepts(question + " " + answer)
        
        # Tạo phần mô tả chính cho hình ảnh
        main_elements = self.generate_visual_elements(concepts)
        
        # Thêm chi tiết về cách trình bày trực quan
        visual_details = "clear visual explanation that helps understand the relationship between the question and answer"
        
        # Lấy phong cách từ thư viện mẫu
        style_description = self.style_templates.get(style, self.style_templates["flat"])
        
        # Tạo prompt cuối cùng
        final_prompt = self.prompt_template.format(
            input_text=input_text,
            style_description=style_description,
            main_elements=main_elements,
            visual_details=visual_details
        )
        
        return final_prompt

    def process_batch(self, input_file, output_file, style="flat"):
        """Xử lý file CSV chứa cặp câu hỏi-trả lời và tạo file prompt"""
        results = []
        
        with open(input_file, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                question = row.get('question', '')
                answer = row.get('answer', '')
                
                if question and answer:
                    prompt = self.generate_prompt(question, answer, style)
                    results.append({
                        'question': question,
                        'answer': answer,
                        'prompt': prompt
                    })
        
        # Lưu kết quả vào file JSON
        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(results, file, ensure_ascii=False, indent=2)
            
        return f"Đã xử lý {len(results)} cặp Q&A và lưu vào {output_file}"


# Sử dụng tool
if __name__ == "__main__":
    generator = QAImagePromptGenerator()
    
    # Ví dụ sử dụng với một cặp Q&A
    question = "Tại sao bầu trời có màu xanh?"
    answer = "Vì ánh sáng xanh bị tán xạ nhiều hơn khi đi qua khí quyển."
    
    prompt = generator.generate_prompt(question, answer, style="infographic")
    print("Prompt mẫu:")
    print(prompt)
    
    # Để xử lý hàng loạt từ file CSV, bỏ comment dòng dưới:
    # result = generator.process_batch('input_qa.csv', 'output_prompts.json', style="flat")
    # print(result)


Prompt mẫu:
Create an educational illustration that explains: Câu hỏi: "Tại sao bầu trời có màu xanh?" Trả lời: "Vì ánh sáng xanh bị tán xạ nhiều hơn khi đi qua khí quyển.". Style: educational infographic style with labeled diagrams and clear visual hierarchy. The image should show core concepts including sáng, xanh?, xanh, trời, nhiều with clear visual explanation that helps understand the relationship between the question and answer.


In [7]:
import openai
import requests
from PIL import Image
from io import BytesIO
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
print(OPENAI_API_KEY[0:5])
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# Prompt dạng chuỗi nhiều dòng
image_prompt = """
You are illustration artist specializing in making flash cards for English learning.
Given a context and a phrase, make a simple illustration that accurately represents its meaning.
---
Context: 'IT - Daily stand up' 
Phrase: 'writing test case.'
"""

# Tạo ảnh
response = client.images.generate(
    model="dall-e-3",
    prompt=image_prompt,
    n=1,
    size="1792x1024"
)

# Lấy URL ảnh
image_url = response.data[0].url

# Tải ảnh về (tuỳ chọn)
image_response = requests.get(image_url)
img = Image.open(BytesIO(image_response.content))
img.save("generated_image.png")


sk-pr


Ver 1: 
```bash
You are illustration artist specializing in making flash cards for English learning.
Given a context and a phrase, make a simple illustration that accurately represents its meaning.
---
Context: 'IT - Daily stand up' 
Phrase: 'writing test case.'
```